In [67]:
# Code below is mainly based on materials at https://cuda.juliagpu.org

In [1]:
Threads.nthreads()

4

In [2]:
N = 2^20
x = fill(1.0f0, N)  # a vector filled with 1.0 (Float32)
y = fill(2.0f0, N)  # a vector filled with 2.0

y .+= x  

1048576-element Vector{Float32}:
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 ⋮
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0

In [5]:
using Test
function sequential_add!(y, x)
    for i in eachindex(y, x)
        @inbounds y[i] += x[i]
    end
    return nothing
end

fill!(y, 2)
sequential_add!(y, x)
@test all(y .== 3.0f0)

Test Passed

In [6]:
function parallel_add!(y, x)
    Threads.@threads for i in eachindex(y, x)
        @inbounds y[i] += x[i]
    end
    return nothing
end

fill!(y, 2)
parallel_add!(y, x)
@test all(y .== 3.0f0)

Test Passed

In [7]:
using BenchmarkTools
@btime sequential_add!($y, $x)

  336.091 μs (0 allocations: 0 bytes)


In [8]:
@btime parallel_add!($y, $x)

  170.101 μs (21 allocations: 2.14 KiB)


In [10]:
using CUDA

In [13]:
CuArray(x[1:3])

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 1.0
 1.0

In [14]:
x_d = CUDA.fill(1.0f0, N)  # a vector stored on the GPU filled with 1.0 (Float32)
y_d = CUDA.fill(2.0f0, N);  # a vector stored on the GPU filled with 2.0

In [15]:
y_d .+= x_d
@test all(Array(y_d) .== 3.0f0)

Test Passed

In [16]:
function add_broadcast!(y, x)
    CUDA.@sync y .+= x
    return
end

add_broadcast! (generic function with 1 method)

In [17]:
@btime add_broadcast!($y_d, $x_d)

  61.794 μs (56 allocations: 1.36 KiB)


In [18]:
function gpu_add1!(y, x)
    for i = 1:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

fill!(y_d, 2)
@cuda gpu_add1!(y_d, x_d)
@test all(Array(y_d) .== 3.0f0)


Test Passed

In [19]:
function bench_gpu1!(y, x)
    CUDA.@sync begin
        @cuda gpu_add1!(y, x)
    end
end

bench_gpu1! (generic function with 1 method)

In [20]:
@btime bench_gpu1!($y_d, $x_d)   # VERY VERY SLOW!

  53.822 ms (268 allocations: 4.45 KiB)


CUDA.HostKernel for gpu_add1!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [21]:
bench_gpu1!(y_d, x_d)  # run it once to force compilation if not precompiled
CUDA.@profile bench_gpu1!(y_d, x_d)

Profiler ran for 53.93 ms, capturing 520 events.

Host-side activity: calling CUDA APIs took 53.28 ms (98.80% of the trace)
┌──────────┬────────────┬───────┬─────────────────────┐
│ Time (%) │ Total time │ Calls │ Name                │
├──────────┼────────────┼───────┼─────────────────────┤
│   98.79% │   53.28 ms │     1 │ cuStreamSynchronize │
│    0.10% │   51.26 µs │     1 │ cuLaunchKernel      │
└──────────┴────────────┴───────┴─────────────────────┘

Device-side activity: GPU was busy for 53.77 ms (99.70% of the trace)
┌──────────┬────────────┬───────┬──────────────────────────────────────────────────────────────┐
│ Time (%) │ Total time │ Calls │ Name                                                         │
├──────────┼────────────┼───────┼──────────────────────────────────────────────────────────────┤
│   99.70% │   53.77 ms │     1 │ _Z9gpu_add1_13CuDeviceArrayI7Float32Li1ELi1EES_IS0_Li1ELi1EE │
└──────────┴────────────┴───────┴────────────────────────────────────────────────

In [22]:
CUDA.@profile trace=true bench_gpu1!(y_d, x_d)  # NOTE single block and single thread

Profiler ran for 53.92 ms, capturing 520 events.

Host-side activity: calling CUDA APIs took 53.24 ms (98.74% of the trace)
┌─────┬───────────┬──────────┬────────┬─────────────────────┐
│  ID │     Start │     Time │ Thread │ Name                │
├─────┼───────────┼──────────┼────────┼─────────────────────┤
│   2 │  41.48 µs │ 33.62 µs │      1 │ cuLaunchKernel      │
│ 518 │ 615.84 µs │ 53.24 ms │      2 │ cuStreamSynchronize │
└─────┴───────────┴──────────┴────────┴─────────────────────┘

Device-side activity: GPU was busy for 53.78 ms (99.73% of the trace)
┌────┬──────────┬──────────┬─────────┬────────┬──────┬──────────────────────────────────────────────────────────────┐
│ ID │    Start │     Time │ Threads │ Blocks │ Regs │ Name                                                         │
├────┼──────────┼──────────┼─────────┼────────┼──────┼──────────────────────────────────────────────────────────────┤
│  2 │ 71.76 µs │ 53.78 ms │       1 │      1 │   16 │ _Z9gpu_add1_13CuDeviceAr

In [23]:
function gpu_add2!(y, x)
    index = threadIdx().x    # this example only requires linear indexing, so just use `x`, there are three dimensions available
    stride = blockDim().x
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

fill!(y_d, 2)
@cuda threads=256 gpu_add2!(y_d, x_d)
@test all(Array(y_d) .== 3.0f0)

Test Passed

In [27]:
fill!(y_d, 2)
@cuda threads=256 gpu_add2!(y_d, x_d)
@test all(Array(y_d) .== 3.0f0)

Test Passed

In [36]:
function bench_gpu2!(y, x)
    CUDA.@sync begin
        @cuda threads=256 gpu_add2!(y, x)
    end
end

bench_gpu2! (generic function with 1 method)

In [37]:
@btime bench_gpu2!($y_d, $x_d)

  1.440 ms (268 allocations: 4.45 KiB)


CUDA.HostKernel for gpu_add2!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [30]:
function gpu_add3!(y, x)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = gridDim().x * blockDim().x
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return
end

numblocks = ceil(Int, N/256)

fill!(y_d, 2)
@cuda threads=256 blocks=numblocks gpu_add3!(y_d, x_d)
@test all(Array(y_d) .== 3.0f0)

Test Passed

In [31]:
function bench_gpu3!(y, x)
    numblocks = ceil(Int, length(y)/256)
    CUDA.@sync begin
        @cuda threads=256 blocks=numblocks gpu_add3!(y, x)
    end
end

bench_gpu3! (generic function with 1 method)

In [32]:
@btime bench_gpu3!($y_d, $x_d)

  58.871 μs (42 allocations: 944 bytes)


CUDA.HostKernel for gpu_add3!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [33]:
CUDA.@profile trace=true bench_gpu3!(y_d, x_d)

Profiler ran for 674.25 µs, capturing 74 events.

Host-side activity: calling CUDA APIs took 593.42 µs (88.01% of the trace)
┌────┬───────────┬───────────┬─────────────────────┐
│ ID │     Start │      Time │ Name                │
├────┼───────────┼───────────┼─────────────────────┤
│  2 │  38.86 µs │ 571.73 µs │ cuLaunchKernel      │
│ 72 │ 666.86 µs │   1.67 µs │ cuStreamSynchronize │
└────┴───────────┴───────────┴─────────────────────┘

Device-side activity: GPU was busy for 44.82 µs (6.65% of the trace)
┌────┬───────────┬──────────┬─────────┬────────┬──────┬──────────────────────────────────────────────────────────────┐
│ ID │     Start │     Time │ Threads │ Blocks │ Regs │ Name                                                         │
├────┼───────────┼──────────┼─────────┼────────┼──────┼──────────────────────────────────────────────────────────────┤
│  2 │ 605.58 µs │ 44.82 µs │     256 │   4096 │   32 │ _Z9gpu_add3_13CuDeviceArrayI7Float32Li1ELi1EES_IS0_Li1ELi1EE │
└────┴─────

In [40]:
kernel = @cuda launch=false gpu_add3!(y_d, x_d)
config = launch_configuration(kernel.fun)
threads = min(N, config.threads)
blocks = cld(N, threads)

1024

In [41]:
fill!(y_d, 2)
kernel(y_d, x_d; threads, blocks)  # kernel is callable
@test all(Array(y_d) .== 3.0f0)

Test Passed

In [42]:
function bench_gpu4!(y, x)
    kernel = @cuda launch=false gpu_add3!(y, x)
    config = launch_configuration(kernel.fun)
    threads = min(length(y), config.threads)
    blocks = cld(length(y), threads)

    CUDA.@sync begin
        kernel(y, x; threads, blocks)
    end
end


bench_gpu4! (generic function with 1 method)

In [43]:
@btime bench_gpu4!($y_d, $x_d)

  60.756 μs (50 allocations: 1.05 KiB)


In [44]:
# Debugging, no println in GPU...
function gpu_add2_print!(y, x)
    index = threadIdx().x    # this example only requires linear indexing, so just use `x`
    stride = blockDim().x
    @cuprintln("thread $index, block $stride")
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

@cuda threads=16 gpu_add2_print!(y_d, x_d)
synchronize()

thread 1, block 16
thread 2, block 16
thread 3, block 16
thread 4, block 16
thread 5, block 16
thread 6, block 16
thread 7, block 16
thread 8, block 16
thread 9, block 16
thread 10, block 16
thread 11, block 16
thread 12, block 16
thread 13, block 16
thread 14, block 16
thread 15, block 16
thread 16, block 16


In [49]:
using CUDA, Adapt

struct Interpolate{A}
    xs::A
    ys::A
end

#Functor - a "callable" data structure.
# Typically we have set of xs and ys and we want to interpolate many times
function (itp::Interpolate)(x)
    i = searchsortedfirst(itp.xs, x)
    i = clamp(i, firstindex(itp.ys), lastindex(itp.ys))
    @inbounds itp.ys[i]
end

# Another way to write the same function (this is however less convenient for vectorization):
function interp2(itp::Interpolate, x)
    i = searchsortedfirst(itp.xs, x)
    i = clamp(i, firstindex(itp.ys), lastindex(itp.ys))
    @inbounds itp.ys[i]
end



xs_cpu = [1.0, 2.0, 3.0]
ys_cpu = [10.0,20.0,30.0]
itp_cpu = Interpolate(xs_cpu, ys_cpu)
pts_cpu = [1.1,2.3]

result_cpu = itp_cpu.(pts_cpu) # calling the faunctor


2-element Vector{Float64}:
 20.0
 30.0

In [57]:
[interp2(itp_cpu, u) for u in pts_cpu]
interp2.(Ref(itp_cpu), pts_cpu) # not convenient vectorization compared to a functor


2-element Vector{Float64}:
 20.0
 30.0

In [61]:
itp = Interpolate(CuArray(xs_cpu), CuArray(ys_cpu))
pts = CuArray(pts_cpu);
try 
  itp.(pts)
catch e
    println(string(e)[1:100])
end

GPUCompiler.KernelError(PTX CompilerJob of MethodInstance for (::GPUArrays.var"#34#36")(::CUDA.CuKer


In [62]:
try 
  interp2.(Ref(itp), pts)
catch e
    println(string(e)[1:100])
end

GPUCompiler.KernelError(PTX CompilerJob of MethodInstance for (::GPUArrays.var"#34#36")(::CUDA.CuKer


In [63]:
isbitstype(typeof(itp))

false

In [64]:
import Adapt
function Adapt.adapt_structure(to, itp::Interpolate)
    xs = Adapt.adapt_structure(to, itp.xs)
    ys = Adapt.adapt_structure(to, itp.ys)
    Interpolate(xs, ys)
end


In [65]:
result = itp.(pts)

2-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 20.0
 30.0

In [66]:
interp2.(Ref(itp), pts)

2-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 20.0
 30.0